In [ ]:
# IMPORTANT: The parameters below are set only for running this notebook independently. 
# When executing the full Ploomber pipeline, these values will be overridden by the settings in `pipeline.yaml`. 
# Any modifications made here will not persist when running the pipeline.


YEAR = 2018
COUNTRY =  'ARM' # Code of the Country
STARTING_YEAR = 2018
upstream = {
    "60_indicator_mapping": {
        "data": f"../data/processed/{COUNTRY}/insight_indicators.xlsx",
    }
}
product = {
    "data": f"../data/processed/{COUNTRY}/indicator_codes.xlsx",
}


data_source = ['data/raw/Standard and Additional Indicators/Standard and Additional Indicators-cp-2018-2021.xlsx', 'data/raw/Standard and Additional Indicators/Standard and Additional Indicators-cp-2022-2025.xlsx']



## Effectiveness - 3
### To what extent UNICEF positioning in the general public, CSOs , and private sector enables the delivery of results for children?


This Notebook produces a spreadsheet summarising indicators and mapping them to goal areas. The indicator code, description, category (standard and additional), unit (percent, number, text, etc.), baseline and target value, baseline and target year, goal areas and performance are included. Two performance metrics are:
1. Finalization of indicator rating (by UNICEF)
2. Indicator progress (based on % of indicator targets achieved) from unicef-ecaro-cpe-dataset.xlsx

In [ ]:
import unicef_cpe
from unicef_cpe.config import PROJ_ROOT
from unicef_cpe.utils import *
from unicef_cpe.genai import *

In [ ]:
country_map = {k:v for k,v in get_ecaro_countries_mapping(priority=False).items() if k in COUNTRY}
country_code_map = {v:k for k,v in country_map.items() }

In [ ]:
df = pd.read_excel(upstream['60_insight_indicators']['data'])

In [ ]:
CODE_MAPPING_PATH_cp_1 = PROJ_ROOT / data_source[0]
CODE_MAPPING_PATH_cp_2 = PROJ_ROOT / data_source[1]
df_map_1 = pd.read_excel(CODE_MAPPING_PATH_cp_1)
df_map_2 = pd.read_excel(CODE_MAPPING_PATH_cp_2)

In [5]:
indicator_goal_area_mapping = df_map_1.set_index('Indicator Code')['Goal Area Description'].to_dict()
indicator_goal_area_mapping.update(df_map_2.set_index('Indicator Code')['Goal Area Description'].to_dict())

indicator_category_mapping = df_map_1.set_index('Indicator Code')['Indicator Category'].to_dict()
indicator_category_mapping.update(df_map_2.set_index('Indicator Code')['Indicator Category'].to_dict())

In [ ]:
df['goal_area'] = df['indicator_code'].replace(indicator_goal_area_mapping)
df['goal_area'] = df['goal_area'].replace({'Special Purpose': 'Cross Sectoral'})

df['country_code'] = df['country'].copy()
df['country'] = df['country'].replace(country_map)
df['indicator_category'] = df['indicator_code'].replace(indicator_category_mapping)


In [ ]:

output_path = Path(product['data'])
output_path.parent.mkdir(parents=True, exist_ok=True)  # Create missing directories
df.to_excel(product['data'], index=False)


In [8]:
################################################################################################################################################################################################